In [61]:
import pandas as pd
import geopandas as gdp

/opt/anaconda3/envs/ads/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [36]:
opt_dir = '../data/raw'
crime = pd.read_excel(f"{opt_dir}/external_data/recorded_offences.xlsx", sheet_name="Table 03", nrows=74721)
crime

In [106]:
crime = crime[['Postcode', 'Offence Count']]
crime_agg_yr_pc = crime.groupby(['Postcode'], as_index=False).sum()
crime_agg_yr_pc = crime_agg_yr_pc.rename(columns={'Postcode' : 'postcode'})
crime_agg_yr_pc

,postcode,Offence Count
0,3000,37272
1,3002,2429
2,3003,2708
3,3004,2415
4,3005,174
...,...,...
691,3990,5
692,3991,118
693,3992,206
694,3995,2179


In [107]:
population = pd.read_csv('../data/raw/external_data/population_complete.csv', index_col=0)
population = population.drop(columns=['ERP 2020', 'Area km2', 'density 2021 persons/km2'])

In [117]:
postcode = pd.read_csv('../data/raw/external_data/australian_postcodes.csv')
postcode = postcode[postcode['state'] == 'VIC']
postcode_sa2 = postcode[['postcode', 'SA2_MAINCODE_2016']]
postcode_sa2 = postcode_sa2.drop_duplicates()
postcode_sa2 = postcode_sa2.rename(columns={'SA2_MAINCODE_2016' : 'SA2 code'})

In [125]:
postcode_population = pd.merge(postcode_sa2, population, on='SA2 code', how='left')
postcode_population_agg = postcode_population.groupby('postcode', as_index=False).sum()
postcode_population_agg

,postcode,SA2 code,ERP 2021
0,3000,206041122.0,47192.0
1,3001,206041122.0,47192.0
2,3002,206041119.0,5803.0
3,3003,206041127.0,3.0
4,3004,412082251.0,32825.0
...,...,...,...
742,8111,206041122.0,47192.0
743,8120,206041122.0,47192.0
744,8205,206041122.0,47192.0
745,8785,212041311.0,32936.0


In [126]:
crime_sa2 = pd.merge(crime_agg_yr_pc, postcode_population_agg, on='postcode', how='left')

In [128]:
crime_sa2['scaled_crime_rate'] = crime_sa2['Offence Count'] / crime_sa2['ERP 2021']

In [131]:
crime_sa2.to_csv("../data/raw/external_data/crime_completed.csv")